In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

import dataset_compute
import snapshot_obj

In [ ]:
importlib.reload(dataset_compute)
importlib.reload(snapshot_obj)

In [ ]:
data = {"plain-LCDM": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [2,1], [3,0], [4,0], [5,0]]), "SimID": "V1_MR_fix"},
        "curv-p082": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [2,1], [3,0], [4,0], [5,0]]), "SimID": "V1_MR_curvaton_p082_fix"}}

In [ ]:
data = {"plain-LCDM-LR": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [2,1], [3,0], [4,0], [5,0]]), "SimID": "V1_LR_fix"},
        "curv-p082-LR": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [2,1], [3,0], [4,0], [5,0]]), "SimID": "V1_LR_curvaton_p082_fix"},
        "curv-p084-LR": {"Groups": np.array([[1,0], [1,1], [1,2], [2,0], [2,1], [3,0], [4,0], [5,0]]), "SimID": "V1_LR_curvaton_p084_fix"}}

In [ ]:
snap_id = 126
for key, sim_data in data.items():
    print(key)
    sim_id = sim_data["SimID"]
    snap = snapshot_obj.Snapshot(sim_id, snap_id)
    cops = snap.get_subhalos("CentreOfPotential")
    coords = dataset_compute.group_particles_by_subhalo(snap, "Coordinates")["Coordinates"]
    
    cops_sel = []
    coords_sel = []
    for group in sim_data["Groups"]:
        idx = snap.index_of_halo(group[0], group[1])
        cops_sel.append(cops[idx] * units.cm.to(units.Mpc))
        coords_sel.append(coords[idx] * units.cm.to(units.Mpc))
    
    data[key]["CentreOfPotential"] = cops_sel
    data[key]["Coordinates"] = coords_sel

In [ ]:
filename = 'main_halo_coords.png'
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", "LowResolution")
filename = os.path.join(path, filename)

In [ ]:
projs = [["x", "y", 0, 1],
         ["x", "z", 0, 2]]

fig, axes = plt.subplots(sharey='row', sharex='col', figsize=(10,5.8),# figsize=(7.5,6.5),
                         ncols=len(data), nrows=len(projs))
plt.subplots_adjust(wspace=0.08)
plt.subplots_adjust(hspace=0.08)


for ax_col in axes.T:        
    ax_col[0].set_xlim(6,11)
    ax_col[0].set_ylim(16,21)
    ax_col[1].set_xlim(6,11)
    ax_col[1].set_ylim(79,84)
    ax_col[-1].set_xlabel("x [Mpc]")
    
axes[0,0].set_ylabel("y [Mpc]")
axes[1,0].set_ylabel("z [Mpc]")

col = [['lightblue', 'blue'],
['pink', 'crimson'],
['gray', 'black'],
['violet', 'darkviolet'],
['lightgreen', 'green'],
['yellow', 'gold'],
['orange', 'darkorange'],
['sandybrown', 'saddlebrown'],
['lightsteelblue', 'steelblue'],
['red', 'darkred']]


# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):

    axes[0, i].set_title(key)
    
    # Iterate through projections (rows):
    for j, ax_set in enumerate(projs):
        
        n_groups = len(sim_data['Groups'])
        legends = [None] * n_groups
        # Iterate through halos:
        n_skip = 1 # only plot every n_skip:th particle
        for idx, k in enumerate(range(n_groups)):
            cop_x = sim_data["CentreOfPotential"][k][ax_set[2]]
            cop_y = sim_data["CentreOfPotential"][k][ax_set[3]]
            part_x = sim_data["Coordinates"][k][::n_skip, ax_set[2]]
            part_y = sim_data["Coordinates"][k][::n_skip, ax_set[3]]
            axes[j,i].scatter(part_x, part_y, c=col[k][0], s=0.1)
            
            # Save items for the legend:
            axes[j,i].scatter(cop_x, cop_y, c=col[k][1], s=0.3)
            legends[idx] = axes[j,i].scatter([], [], c=col[k][0], s=15)
   
labels = ["{}, {}".format(grp[0],grp[1]) for grp in data["plain-LCDM-LR"]["Groups"]]
plt.legend(legends, labels, 
           loc='lower left',
           bbox_to_anchor=(1, 0),
           title="GN, SGN")
# Adjust the scaling factor to fit your legend text completely outside the plot
# (smaller value results in more space being made for the legend)
plt.subplots_adjust(right=0.78)

plt.savefig(filename, dpi=300, bbox_inches='tight')

projs = [["x", "y", 0, 1],
         ["x", "z", 0, 2],
         ["y", "z", 1, 2]]

fig, axes = plt.subplots(sharey='row', ncols=len(data), nrows=len(projs))
plt.subplots_adjust(wspace=0.8)
plt.subplots_adjust(hspace=0.8)


for ax_row in axes.T:        
    ax_row[0].set_xlim(7,10)
    ax_row[0].set_ylim(15,22)
    ax_row[1].set_xlim(7,10)
    ax_row[1].set_ylim(81,83)
    ax_row[2].set_xlim(15,22)
    ax_row[2].set_ylim(81,83)
    

col = [['pink', 'red'], ['lightblue', 'blue'], ['lightgreen', 'green'], ['yellow', 'orange'], ['violet', 'purple'], ['gray', 'black']]

# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):

    axes[0, i].set_title(key)
    
    # Iterate through projections (rows):
    for j, ax_set in enumerate(projs):
        axes[j,i].set_xlabel("${}$".format(ax_set[0]))
        axes[j,i].set_ylabel("${}$".format(ax_set[1]))
        
        n_groups = len(sim_data['Groups'])
        legends = [None] * n_groups
        # Iterate through halos:
        n_skip = 50 # only plot every n_skip:th particle
        for idx, k in enumerate(range(n_groups)):
            cop_x = sim_data["CentreOfPotential"][k][ax_set[2]]
            cop_y = sim_data["CentreOfPotential"][k][ax_set[3]]
            part_x = sim_data["Coordinates"][k][::n_skip, ax_set[2]]
            part_y = sim_data["Coordinates"][k][::n_skip, ax_set[3]]
            axes[j,i].scatter(part_x, part_y, c=col[k][0], s=0.1)
            
            # Save items for the legend:
            axes[j,i].scatter(cop_x, cop_y, c=col[k][1], s=0.1)
            legends[idx] = axes[j,i].scatter([], [], c=col[k][0], s=3)
   
labels = ["{}, {}".format(grp[0],grp[1]) for grp in data["LCDM"]["Groups"]]
plt.legend(legends, labels, 
           bbox_to_anchor=(1.1, 2.0),
           title="GN, SGN")
# Adjust the scaling factor to fit your legend text completely outside the plot
# (smaller value results in more space being made for the legend)
plt.subplots_adjust(right=0.78)

#plt.savefig(filename, dpi=300)

